### 1.准备阶段

1.1 打包容器镜像

In [ ]:
# !sh build_push_lora_v2.sh

1.2 下载数据集

In [21]:
# !wget https://d374aanje223q0.cloudfront.net/pokemon-blip-captions-dataset.tar.gz
# !tar -xzvf pokemon-blip-captions-dataset.tar.gz
# !cp dataset.toml pokemon-blip-captions-dataset/
# !cd pokemon-blip-captions-dataset

# 将数据集上传到S3中
#!aws s3 cp ./ $images_s3uri --recursive
#!aws s3 ls $images_s3uri 


                           PRE .ipynb_checkpoints/
2023-03-16 02:01:06     262251 00000-0-IMG20230302163943.png
2023-03-16 02:01:06        144 00000-0-IMG20230302163943.txt
2023-03-16 02:01:06     269277 00001-0-IMG20230302163945.png
2023-03-16 02:01:06        116 00001-0-IMG20230302163945.txt
2023-03-16 02:01:06     287568 00002-0-IMG20230302163946.png
2023-03-16 02:01:06         95 00002-0-IMG20230302163946.txt
2023-03-16 02:01:06     289849 00003-0-IMG20230302163947.png
2023-03-16 02:01:06        166 00003-0-IMG20230302163947.txt
2023-03-16 02:01:06     275408 00004-0-IMG20230302163948.png
2023-03-16 02:01:06        189 00004-0-IMG20230302163948.txt
2023-03-16 02:01:06     267037 00005-0-IMG20230302163954.png
2023-03-16 02:01:06        169 00005-0-IMG20230302163954.txt
2023-03-16 02:01:06     296417 00006-0-IMG20230302163956.png
2023-03-16 02:01:06        115 00006-0-IMG20230302163956.txt
2023-03-16 02:01:06        270 dataset.toml


### 2.训练阶段

In [14]:
import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

print("bucket: {0}".format(bucket))
print("role: {0}".format(role))
print("account_id: {0}".format(account_id))
print("region_name: {0}".format(region_name))

bucket: sagemaker-us-east-1-091063646508
role: arn:aws:iam::091063646508:role/service-role/AmazonSageMaker-ExecutionRole-20230219T124484
account_id: 091063646508
region_name: us-east-1


In [15]:
import json
from sagemaker.estimator import Estimator

def json_encode_hyperparameters(hyperparameters):
    for (k, v) in hyperparameters.items():
        print(k, v)

    return {k: json.dumps(v) for (k, v) in hyperparameters.items()}

# 1.设置基础模型
# model_name_or_path="runwayml/stable-diffusion-v1-5"
model_name_or_path="s3://{0}/lora-pretrain-model/chilloutmix_NiPrunedFp32Fix.safetensors".format(bucket)


# 2.设置训练数据集
images_s3uri = 's3://{0}/train/datasets/20230425/'.format(bucket)

# 3.设置容器镜像地址
image_uri = '091063646508.dkr.ecr.us-east-1.amazonaws.com/lora-finetuning-v2:latest'

# 4.设置用于训练的机型
instance_type = 'ml.g4dn.xlarge'

# 5.设置输出目录，环境变量，超参
output_dir="/opt/ml/model/"
lora_model_name="aws-trained-lora-model"

inputs = {
    'images': images_s3uri
}
# 如果基础模型是S3的地址，直接映射到容器中 /opt/ml/input/models 目录下
pretrained_model_name_or_path=model_name_or_path
if model_name_or_path.startswith("s3://"):
    inputs['models'] = model_name_or_path
    pretrained_model_name_or_path = "/opt/ml/input/data/models/" + model_name_or_path.split('/')[-1]

print(pretrained_model_name_or_path)


/opt/ml/input/data/models/chilloutmix_NiPrunedFp32Fix.safetensors


In [16]:
environment = {
    'PYTORCH_CUDA_ALLOC_CONF':'max_split_size_mb:32',
    'LD_LIBRARY_PATH':"${LD_LIBRARY_PATH}:/opt/conda/lib/"
}

hyperparameters = {
                    'mixed_precision':'fp16',
                    'pretrained_model_name_or_path': pretrained_model_name_or_path,
                    'dataset_config':'/opt/ml/input/data/images/dataset.toml',
                    'output_dir':output_dir,
                    'output_name':lora_model_name,
                    'save_model_as':'safetensors', 
                    'prior_loss_weight':1.0, 
                    'max_train_steps':400, 
                    'learning_rate':1e-4, 
                    'optimizer_type':"AdamW8bit", 
                    'xformers':True, 
                    'mixed_precision':"fp16", 
                    'cache_latents':True, 
                    'gradient_checkpointing':True,
                    'save_every_n_epochs':100, 
                    'network_module':'networks.lora'
}

hyperparameters = json_encode_hyperparameters(hyperparameters)

# 6.使用 Spot Instance 开始训练
estimator = Estimator(
    role = role,
    instance_count=1,
    use_spot_instances=True,
    max_wait=7200,
    max_run=3600,
    instance_type = instance_type,
    image_uri = image_uri,
    hyperparameters = hyperparameters,
    environment = environment
)
estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: lora-finetuning-v2-2023-04-26-03-29-49-073


mixed_precision fp16
pretrained_model_name_or_path /opt/ml/input/data/models/chilloutmix_NiPrunedFp32Fix.safetensors
dataset_config /opt/ml/input/data/images/dataset.toml
output_dir /opt/ml/model/
output_name aws-trained-lora-model
save_model_as safetensors
prior_loss_weight 1.0
max_train_steps 400
learning_rate 0.0001
optimizer_type AdamW8bit
xformers True
cache_latents True
gradient_checkpointing True
save_every_n_epochs 100
network_module networks.lora
2023-04-26 03:29:51 Starting - Starting the training job...
2023-04-26 03:30:06 Starting - Preparing the instances for training...
2023-04-26 03:30:58 Downloading - Downloading input data...............
2023-04-26 03:33:09 Training - Downloading the training image.....................
2023-04-26 03:37:00 Training - Training image download completed. Training in progress...2023-04-26 03:37:11,478 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-04-26 03:37:11,514 sagemaker-training-toolkit I

### 3.推理阶段

In [ ]:
!pip install --upgrade diffusers accelerate transformers safetensors omegaconf

将训练后的模型下载到本地，并解压缩

In [ ]:
lora_model_data = estimator.model_data
print("Model artifact saved at:\n", lora_model_data)

!aws s3 cp $lora_model_data ./
!tar -xvf model.tar.gz
!rm -fr model.tar.gz

如果基础模型是上传到S3的safetensors文件，则将基础模型下载到本地

In [5]:
import boto3
import torch
from diffusers.pipelines.stable_diffusion.convert_from_ckpt import download_from_original_stable_diffusion_ckpt

# 将 safetensors 文件转成 diffuser 格式
def convert_from_safetensors_to_diffuser(checkpoint_path,dump_path):
    pipe = download_from_original_stable_diffusion_ckpt(
            checkpoint_path=checkpoint_path,
            original_config_file=None,
            image_size=None,
            prediction_type=None,
            model_type=None,
            extract_ema=True,
            scheduler_type='pndm',
            num_in_channels=None,
            upcast_attention=True,
            from_safetensors=True,
            device='cuda:0',
            stable_unclip=None,
            stable_unclip_prior=None,
            clip_stats_path=None,
            controlnet=None,
        )
    pipe.to(torch_dtype=torch.float16)
    pipe.save_pretrained(dump_path, safe_serialization=True)

    
if model_name_or_path.startswith("s3://"):
    s3_client = boto3.client("s3")
    s3_path = model_name_or_path[len("s3://"):]
    bucket = s3_path.split('/')[0]
    key = s3_path[len(bucket)+1:]
    file_name = s3_path.split("/")[-1]
    #print("s3_path:{0}".format(s3_path))
    #print("bucket:{0}".format(bucket))
    #print("key:{0}".format(key))
    #print("file_name:{0}".format(file_name))
    s3_client.download_file(bucket,key,file_name)

    # 将基础模型转换成 Diffuser 格式
    base_model_diffuser_dir=file_name.split(".safetensors")[0]
    convert_from_safetensors_to_diffuser(file_name,base_model_diffuser_dir)
    model_name_or_path = model_name_or_path
    

s3_path:sagemaker-us-east-1-091063646508/lora-pretrain-model/chilloutmix_NiPrunedFp32Fix.safetensors,bucket:sagemaker-us-east-1-091063646508,key:lora-pretrain-model/chilloutmix_NiPrunedFp32Fix.safetensors,file_name:chilloutmix_NiPrunedFp32Fix.safetensors


定义函数，加载lora权重

In [ ]:
from safetensors.torch import load_file
from collections import defaultdict

def load_lora_weights(pipeline, checkpoint_path, multiplier, device, dtype):
    LORA_PREFIX_UNET = "lora_unet"
    LORA_PREFIX_TEXT_ENCODER = "lora_te"
    # load LoRA weight from .safetensors
    state_dict = load_file(checkpoint_path, device=device)

    updates = defaultdict(dict)
    for key, value in state_dict.items():
        # it is suggested to print out the key, it usually will be something like below
        # "lora_te_text_model_encoder_layers_0_self_attn_k_proj.lora_down.weight"

        layer, elem = key.split('.', 1)
        updates[layer][elem] = value

    # directly update weight in diffusers model
    for layer, elems in updates.items():

        if "text" in layer:
            layer_infos = layer.split(LORA_PREFIX_TEXT_ENCODER + "_")[-1].split("_")
            curr_layer = pipeline.text_encoder
        else:
            layer_infos = layer.split(LORA_PREFIX_UNET + "_")[-1].split("_")
            curr_layer = pipeline.unet

        # find the target layer
        temp_name = layer_infos.pop(0)
        while len(layer_infos) > -1:
            try:
                curr_layer = curr_layer.__getattr__(temp_name)
                if len(layer_infos) > 0:
                    temp_name = layer_infos.pop(0)
                elif len(layer_infos) == 0:
                    break
            except Exception:
                if len(temp_name) > 0:
                    temp_name += "_" + layer_infos.pop(0)
                else:
                    temp_name = layer_infos.pop(0)

        # get elements for this layer
        weight_up = elems['lora_up.weight'].to(dtype)
        weight_down = elems['lora_down.weight'].to(dtype)
        alpha = elems['alpha']
        if alpha:
            alpha = alpha.item() / weight_up.shape[1]
        else:
            alpha = 1.0

        # update weight
        if len(weight_up.shape) == 4:
            curr_layer.weight.data += multiplier * alpha * torch.mm(weight_up.squeeze(3).squeeze(2), weight_down.squeeze(3).squeeze(2)).unsqueeze(2).unsqueeze(3)
        else:
            curr_layer.weight.data += multiplier * alpha * torch.mm(weight_up, weight_down)

    return pipeline

In [ ]:
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained(model_name_or_path, torch_dtype=torch.float16)

pipe.to("cuda")

pipe = load_lora_weights(pipe, lora_model_name+".safetensors", 0.8, 'cuda', torch.float16)


In [ ]:
# 执行推理方法
image = pipe("a anime boy,YanJun", num_inference_steps=25).images[0]
image.show()